In [1]:
import math
import sys
print(sys.executable)

import datasets
import matplotlib.pyplot as plt
import numpy as np
import rich
import rich.table
import torch
import transformers


/home/mila/g/gagnonju/.main/bin/python


In [2]:
table = rich.table.Table()

In [3]:
MODEL_NAME = "gpt2"
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16, device_map={"": 0}, pad_token_id=tokenizer.pad_token_id)

[2023-08-29 17:22:59,092] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
DEFAULTS = dict(
    max_new_tokens = 70,
    do_sample=True,
    return_dict=True
)

def generate(text, **kwargs):
    args = DEFAULTS.copy()
    args.update(kwargs)
    
    tokens = tokenizer(text, return_tensors="pt", padding=True).to(0)
    model_output = model.generate(
        **tokens, 
        **kwargs, 
        return_dict_in_generate=True, 
        output_scores=True,
    )
    gen_seq_len = len(model_output["scores"])
    gen_seq = model_output['sequences'][:, -gen_seq_len:]
    scores = torch.cat(model_output["scores"]).softmax(-1)
    print(f"{gen_seq_len = }")
    print(f"{gen_seq.shape = }")
    winning_scores = torch.gather(scores, dim=1, index=gen_seq)
    print(f"{winning_scores.shape = }")
    print(f"{model_output['sequences'].shape = }")
    return model_output["sequences"][0, -gen_seq_len:], [x.item() for x in winning_scores[0]]

In [37]:
tokens, scores = generate("Lol this is a test")

gen_seq_len = 14
gen_seq.shape = torch.Size([1, 14])
winning_scores.shape = torch.Size([1, 14])
model_output['sequences'].shape = torch.Size([1, 20])


In [52]:
import rich.console
console = rich.console.Console(width=240)
table = rich.table.Table(show_header=False, show_lines=True)
print([int(x * 255) for x in scores])

def to_col(x):
    def to_val(y):
        return int(192 * y) + 192

    val = to_val(x)
    return f"rgb({val},{val},{val})"

def apply_col(text, score):
    return f"[black on {to_col(score)}]{text}"


table.add_row(*[apply_col(tokenizer.decode(x), score) for x, score in mit.zip_equal(tokens, scores)])
table.add_row(*[apply_col(x, x) for x in scores])
console.print(table)

[48, 0, 0, 48, 0, 0, 10, 1, 1, 0, 0, 0, 0, 0]


┌─────┬──────┬────────┬─────┬──────┬───────┬────┬───┬───┬─────┬───────┬─────┬──────┬───────┐
│  of │  the │  power │  of │  the │  mind │ .  │   │   │ The │  mind │  is │  the │  most │
│     │      │        │     │      │       │    │   │   │     │       │     │      │       │
├─────┼──────┼────────┼─────┼──────┼───────┼────┼───┼───┼─────┼───────┼─────┼──────┼───────┤
│ 48  │ 0    │ 0      │ 48  │ 0    │ 0     │ 10 │ 1 │ 1 │ 0   │ 0     │ 0   │ 0    │ 0     │
└─────┴──────┴────────┴─────┴──────┴───────┴────┴───┴───┴─────┴───────┴─────┴──────┴───────┘

In [51]:
256-64

192